🎯 Yêu cầu:
Thiết kế một cấu trúc LFUCache hỗ trợ hai hàm:

python
Sao chép
Chỉnh sửa
get(key) -> int
put(key, value) -> None
Trong đó:

get(key):

Nếu key tồn tại, trả về giá trị tương ứng

Tăng tần suất sử dụng của key đó lên 1

put(key, value):

Thêm key mới (nếu chưa có)

Nếu vượt quá capacity → xóa phần tử có tần suất nhỏ nhất

Nếu có nhiều phần tử cùng tần suất nhỏ nhất → xóa cái ít được sử dụng gần đây nhất

⛔ Mục tiêu:
Tất cả thao tác phải chạy trong O(1) thời gian!

In [ ]:
from collections import defaultdict

# Node của danh sách liên kết đôi
class Node:
    def __init__(self, key, val):
        self.key = key        # khóa của node
        self.val = val        # giá trị
        self.freq = 1         # tần suất truy cập ban đầu
        self.prev = None
        self.next = None

In [ ]:
# Danh sách liên kết đôi với dummy head & tail
class DoublyLinkedList:
    def __init__(self):
        self.head = Node(None, None)   # dummy head
        self.tail = Node(None, None)   # dummy tail
        self.head.next = self.tail
        self.tail.prev = self.head

    def is_empty(self):
        return self.head.next == self.tail

    def append(self, node):
        # Thêm node vào cuối (trước tail)
        node.prev = self.tail.prev
        node.next = self.tail
        self.tail.prev.next = node
        self.tail.prev = node

    def remove(self, node):
        # Gỡ bỏ node khỏi danh sách
        node.prev.next = node.next
        node.next.prev = node.prev

    def pop_left(self):
        # Xóa node đầu tiên (cũ nhất trong freq)
        if self.is_empty():
            return None
        node = self.head.next
        self.remove(node)
        return node

In [ ]:
# Lớp LFU Cache
class LFUCache:
    def __init__(self, capacity):
        self.capacity = capacity        # dung lượng cache
        self.size = 0                   # số phần tử hiện tại
        self.min_freq = 0               # tần suất nhỏ nhất
        self.key_table = {}             # key -> Node
        self.freq_table = defaultdict(DoublyLinkedList)  # freq -> DLL chứa các node cùng freq

    def _update_freq(self, node):
        """Cập nhật tần suất của node"""
        freq = node.freq

        # Xóa node khỏi danh sách tần suất cũ
        self.freq_table[freq].remove(node)

        # Nếu danh sách cũ rỗng và là min_freq → tăng min_freq
        if self.freq_table[freq].is_empty():
            del self.freq_table[freq]
            if self.min_freq == freq:
                self.min_freq += 1

        # Tăng tần suất và thêm node vào danh sách mới
        node.freq += 1
        self.freq_table[node.freq].append(node)

    def get(self, key):
        if key not in self.key_table:
            return -1

        node = self.key_table[key]
        self._update_freq(node)   # vì truy cập → tăng tần suất
        return node.val

    def put(self, key, value):
        if self.capacity == 0:
            return  # cache không có dung lượng

        if key in self.key_table:
            # Nếu key đã tồn tại → cập nhật value và tăng freq
            node = self.key_table[key]
            node.val = value
            self._update_freq(node)
        else:
            if self.size == self.capacity:
                # Xóa node ít dùng nhất (theo min_freq)
                lfu_list = self.freq_table[self.min_freq]
                node_to_remove = lfu_list.pop_left()
                del self.key_table[node_to_remove.key]
                self.size -= 1

            # Thêm node mới
            new_node = Node(key, value)
            self.key_table[key] = new_node
            self.freq_table[1].append(new_node)
            self.min_freq = 1       # reset min_freq khi có node mới
            self.size += 1


In [ ]:
lfu = LFUCache(2)
lfu.put(1, 1)
lfu.put(2, 2)
print(lfu.get(1))  # 1 (freq[1]=2)
lfu.put(3, 3)      # Xoá key 2 (vì freq = 1)
print(lfu.get(2))  # -1
print(lfu.get(3))  # 3
print(lfu.get(1))  # 1
